In [1]:
from keras.preprocessing import sequence
import pandas as pd
import numpy as np

/home/huypham/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pickle

df = pickle.load(open('train_encode.p', 'rb'))

In [3]:
df.head()

,descVec,wordIdVec
0,"[0, 2430, 3956, 4690, 2419, 2340, 775, 43, 488...","[0, 2340, 1]"
1,"[0, 2430, 3212, 3957, 4690, 3, 3733, 4948, 43,...","[0, 3733, 1]"
2,"[0, 2430, 3212, 3957, 4690, 2198, 2436, 3327, ...","[0, 2594, 1]"
3,"[0, 2430, 5291, 4812, 4096, 5338, 3256, 875, 3...","[0, 3036, 1]"
4,"[0, 2802, 2034, 383, 3415, 775, 186, 43, 2064,...","[0, 2802, 1]"


In [4]:
max([len(i) for i in df.descVec])

50

In [5]:
max([len(i) for i in df.wordIdVec])

10

In [6]:
maxlen = 50
max_features = 5356

In [7]:
encoder_input_data = np.zeros(
    (len(df), 50),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(df), 10),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(df), 10, max_features),
    dtype='float32')

In [8]:
for i, (input_text, target_text) in enumerate(zip(df.descVec[:5], df.wordIdVec[:5])):
    for t, word in enumerate(input_text):
        encoder_input_data[i, t] = word
    for t, word in enumerate(target_text):
        decoder_input_data[i, t] = word
        if t > 0:
            decoder_target_data[i, t - 1, word] = 1.

In [9]:
decoder_input_data[0]

array([0.00e+00, 2.34e+03, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00], dtype=float32)

# Example of generate data
for i, (input_text, target_text) in enumerate(zip(df.descVec, df.wordIdVec)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [10]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

embedding_size = 50

In [11]:
# Encoder model
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(max_features, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [12]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dex=  Embedding(max_features, embedding_size)
final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state=encoder_states)
decoder_dense = Dense(max_features, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [13]:
from keras.optimizers import SGD
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     267800      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     267800      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [15]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=32,
          epochs=10,
          validation_split=0.05)

Train on 9166 samples, validate on 483 samples
Epoch 1/10
9166/9166 [==============================] - 109s 12ms/step - loss: 9.3639e-04 - acc: 5.4549e-05 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
9166/9166 [==============================] - 111s 12ms/step - loss: 9.3629e-04 - acc: 5.4549e-05 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 3/10
9166/9166 [==============================] - 114s 12ms/step - loss: 9.3619e-04 - acc: 5.4549e-05 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 4/10
9166/9166 [==============================] - 120s 13ms/step - loss: 9.3608e-04 - acc: 5.4549e-05 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 5/10
9166/9166 [==============================] - 151s 16ms/step - loss: 9.3598e-04 - acc: 5.4549e-05 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 6/10
9166/9166 [==============================] - 116s 13ms/step - loss: 9.3588e-04 - acc: 5.4549e-05 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 7/10
9166/9166 [=============